<a href="https://colab.research.google.com/github/nagym72/machine-learning-implementations-with-pytorch/blob/main/xai_project_2_alexnet_skin_data_gradcam_error_corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skin Data

In [ ]:
# necessary so that we don't run into problems while downloading data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import pandas as pd
import os
import time
from pathlib import Path
import torch
import torchvision
from torchvision import transforms
from torchvision import models
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
#new dazugekommen
from torch.utils.data.dataloader import default_collate
#
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import trange
from sklearn.utils import shuffle
import io

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hmnist_28_28_RGB.csv to hmnist_28_28_RGB.csv


In [ ]:
RANDOM_SEED = 42

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
pip install medcam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 52.7 MB 226 kB/s 
  Created wheel for medcam: filename=medcam-0.1.21-py3-none-any.whl size=23214 sha256=9cac0c84ed3ff04013e5b0e9b2acc6e013f0cef7fb7788f9609da2ece36e7c2b
  Stored in directory: /root/.cache/pip/wheels/98/da/39/4705b259cd88818af023a6021dbbbe5e97863b2fdc9b965109
Successfully built medcam


In [ ]:
def get_skin_loader(root, train=True, batch_size=1, num_workers=0, shuffle=False):

    #pandas df labels are read in as np.int32
    df_skin = pd.read_csv(root, dtype={"label": np.int32})
    #subsample of pandas df
    df_skin = df_skin.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    print(f'number of images: {len(df_skin)}')
    
    #conversion to np.float, drop of "label" column
    np_rgb = df_skin.drop('label', axis=1).to_numpy(dtype=np.float32)
    #reshape to nx28x28x3
    images = np.reshape(np_rgb,(len(np_rgb), 28, 28, 3))
    
    # transform images into desired shape: from (28,28,3) to (3,224,224)
    # images = np.moveaxis(np.repeat(np.repeat(images, repeats=8, axis=1), repeats=8, axis=2), -1, 1)
    
    # reshape images to (3,28,28), later in the dataloader
    images = np.moveaxis(images, -1, 1)
    
    print('images loaded and resized')
    images = images/255
    print('images normalized')
    #convert to torch.tensor.float32
    images = torch.from_numpy(images)
    print(images.dtype)
    # add labels and create data as tuples
    #dtype:  images: numpy.float array, label:str

    print(df_skin["label"][0].dtype)
    print(type(images))
    
    image_labels_list = list(zip(images, df_skin['label'].to_numpy()))
    print('images zipped')
    

    size_train = int(len(images) * 0.9)
    size_val = int(len(images) - size_train)
    train_set, val_set = random_split(image_labels_list, [size_train, size_val])

    train_loader = DataLoader(train_set, batch_size = batch_size, num_workers = num_workers, shuffle = shuffle)
    val_loader = DataLoader(val_set, batch_size = batch_size, num_workers = num_workers, shuffle = shuffle)

    return train_loader, val_loader

[Errno 2] No such file or directory: 'C:\\Users\\adm2\\Desktop\\Michael\\HAM10000'
/content


In [ ]:
train_set, val_set = get_skin_loader("hmnist_28_28_RGB.csv",train=True, batch_size=50, num_workers=0, shuffle=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print("--------------------------------------------------------------------")


# get some random training images
dataiter = iter(train_set)
images, labels = next(dataiter)

#print(images, labels)
#images are torch.float32
#labels are torch.int32

#images and labels are on CPU! should be on cuda 
images.device
labels.device


number of images: 10015
images loaded and resized
images normalized
torch.float32
int32
<class 'torch.Tensor'>
images zipped
--------------------------------------------------------------------


device(type='cpu')

#### Skin data

In [ ]:
class project2(nn.Module):
    def __init__(self, model_trained: nn.Module, num_classes: int = 10):
        
        super(project2, self).__init__()
        self.num_classes = num_classes
        self.classifier = nn.Linear(1000, num_classes)
        self.model_trained = model_trained

    def forward(self, x):
        x = self.model_trained(x)
        x.to(device)
        x = self.classifier(x)
        return x

    def printer(self):
        print(self.num_classes)
        print(self.classifier)
        print(self.model_trained)

        



In [ ]:
def _forward(network: nn.Module, data: DataLoader, metric: callable, give_logits=False, augment_data=False):
    device = next(network.parameters()).device
    
    #transforms for the image
    transform = transforms.Compose([
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomResizedCrop(size=(28, 28), scale=(0.7, 1)),
    ])
    
    #if give_logits:
    for x, y in data:
        if augment_data and np.random.uniform()>0.5:
            x = transform(x)
        x = np.repeat(np.repeat(x, repeats=8, axis=2), repeats=8, axis=3)

        #converting to torch.arrays, push to GPU
        x, y = x.to(device, dtype=torch.float), y.to(device, dtype=torch.long)
        #x = torch.float
        #y = torch.long


        logits = network(x)
        res = metric(logits, y)
        if give_logits:
            yield res, torch.argmax(logits.detach().clone(), dim=1).tolist(), y.detach().clone().tolist()
        else:
            yield res
    #else:
        #for x, y in data:
            #x = np.repeat(np.repeat(x, repeats=8, axis=2), repeats=8, axis=3)
            #x, y = x.to(device, dtype=torch.float), y.to(device, dtype=torch.long)
            #logits = network(x)
            #res = metric(logits, y)
            #yield res

@torch.no_grad()
def evaluate(network: nn.Module, data: DataLoader, metric: callable, give_logits=False) -> list:
    network.eval()
    if give_logits:
        yielded_results = _forward(network, data, metric, give_logits=True)        
        return [(res[0].item(), res[1], res[2]) for res in yielded_results]
    else:
        results = _forward(network, data, metric)
        return [res.item() for res in results]

In [ ]:
@torch.enable_grad()
def update(network: nn.Module, data: DataLoader, loss: nn.Module,
           opt: optim.Optimizer, augment_data=True) -> list:
    network.train()
    
    errs = []
    for err in _forward(network, data, loss, augment_data=augment_data):
        errs.append(err.item())
        opt.zero_grad()
        err.backward()
        opt.step()
        
    return errs

In [ ]:
@torch.no_grad()
def accuracy(logits, targets):

    top_pred = logits.argmax(1, keepdim = True)
    correct = top_pred.eq(targets.view_as(top_pred)).sum()
    acc = correct.float() / targets.shape[0]

    return acc

In [ ]:
model_pretrained = models.alexnet(weights=True)
# model_pretrained = models.resnet50(pretrained=True)
#model_pretrained = models.resnet152(pretrained=True)
for param in model_pretrained.parameters():
    param.requires_grad = False

model_project2 = project2(model_pretrained, num_classes = 7)
model_project2.eval()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


project2(
  (classifier): Linear(in_features=1000, out_features=7, bias=True)
  (model_trained): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
   

In [ ]:
def generate_data(model, epochs:int, lr:float, use_augmented_data:bool, model_name:str,
                  train_set=train_set, val_set=val_set, device=device, to_folder='epoch_data_all_samples'):
    
    #summaries are going to be stored in a pd.DataFrame
    df_data = pd.DataFrame(columns=['name', 'epoch', 'lr', 'classifier.weight', 'classifier.bias',
                                'confusion_matrix', 'train_errs', 'valid_errs'])
    
    #push to GPU
    model_project2.to(device)
    #push to GPU 
    device = next(model_project2.parameters()).device

    # optimiser + loss function
    optim = torch.optim.Adam(model_project2.parameters(), lr = lr)
    ce = nn.CrossEntropyLoss()

    name_list, epoch_list, lr_list = [], [], []
    conf_matrix_list, train_errs_list, valid_errs_list = [], [], []

    
    weights_shape = np.zeros_like(dict([i for i in model_project2.named_parameters()])[
        'classifier.weight'].detach().cpu().numpy()).shape


    bias_shape = np.zeros_like(dict([i for i in model_project2.named_parameters()])[
        'classifier.bias'].detach().cpu().numpy()).shape


    weights_array, bias_array = np.zeros((epochs,weights_shape[0],weights_shape[1]), dtype=np.float32), np.zeros((epochs,bias_shape[0]), dtype=np.float32)

    #delete again
    logits_argmax_list = None
    y_list = None
    val_errs = None
    conf_matrix = None

    for i in trange(epochs):
        df_data = pd.DataFrame(columns=['name', 'epoch', 'lr', 'classifier.weight', 'classifier.bias',
                                    'confusion_matrix', 'train_errs', 'valid_errs', 'used_data_augmentation'])
        start_time = time.time()

        # training 
        local_errs = update(model_project2, train_set, ce, optim, augment_data=True)
        # training
        train_errs_list.append(sum(local_errs) / len(local_errs))

        # validation
        val_output = np.array(evaluate(model_project2, val_set, ce, give_logits=True))
        val_errs = val_output[:,0]
        valid_errs_list.append((sum(val_errs) / len(val_errs)))

        # logits and y for confusion matrix
        logits_argmax_list = val_output[:,1]
        y_list = val_output[:,2]
        conf_matrix = confusion_matrix(np.array(sum(y_list.tolist(), [])), np.array(sum(logits_argmax_list.tolist(), [])))
        conf_matrix_list.append(conf_matrix)

        acc_train = evaluate(model_project2, train_set, metric = accuracy)
        acc_val = evaluate(model_project2, val_set, metric = accuracy)

        name_list.append('alexnet_1')
        epoch_list.append(i)
        lr_list.append(lr)

        # add model parameters, but only the last layer
        weights_array[i] = dict([i for i in model_project2.named_parameters()])['classifier.weight'].detach().cpu().numpy()
        bias_array[i] = dict([i for i in model_project2.named_parameters()])['classifier.bias'].detach().cpu().numpy()

        # safe the data for each epoch
        df_data['name'], df_data['epoch'], df_data['lr'] = name_list, epoch_list, lr_list
        df_data['classifier.weight'],df_data['classifier.bias'],df_data['confusion_matrix'] = list(weights_array)[:i+1], list(bias_array)[:i+1], conf_matrix_list
        df_data['train_errs'], df_data['valid_errs'] = train_errs_list, valid_errs_list
        df_data['used_data_augmentation'] = [use_augmented_data]*len(df_data)
        
        # safe current dataframe after each epoch
        df_data.to_pickle(f'{to_folder}\\{model_name}_epoch_{i+1}')
        
        print("Accuracy on train data: {:.2f} %".format(sum(acc_train) / len(acc_train) * 100))
        print("Accuracy on validation data: {:.2f} %".format(sum(acc_val) / len(acc_val) * 100))
        print(f'--- {round(time.time()-start_time, 2)} seconds ---')
    
    return df_data

In [ ]:
df_alexnet_1 = generate_data(model_pretrained, epochs=3, lr=1e-4, use_augmented_data=False, model_name='alexnet_1')

  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-60-dfae8716fbd0>:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val_output = np.array(evaluate(model_project2, val_set, ce, give_logits=True))


Accuracy on train data: 68.36 %
Accuracy on validation data: 67.81 %
--- 22.71 seconds ---


<ipython-input-60-dfae8716fbd0>:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val_output = np.array(evaluate(model_project2, val_set, ce, give_logits=True))


Accuracy on train data: 68.89 %
Accuracy on validation data: 68.67 %
--- 19.72 seconds ---


<ipython-input-60-dfae8716fbd0>:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val_output = np.array(evaluate(model_project2, val_set, ce, give_logits=True))


Accuracy on train data: 69.14 %
Accuracy on validation data: 68.67 %
--- 21.97 seconds ---


In [ ]:
from medcam import medcam

In [ ]:
batch = next(iter(train_set))

for entr in batch:
  print(entr.shape)


#input fuer das model ist nun 50,3,28,28
#
print("Struktur des Netzwerkes ist:\n")
model_project2.eval()




torch.Size([50, 3, 28, 28])
torch.Size([50])
Struktur des Netzwerkes ist:



project2(
  (classifier): Linear(in_features=1000, out_features=7, bias=True)
  (model_trained): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
   

In [ ]:
model = medcam.inject(model_project2, output_dir='attention_maps', backend='gcam', layer='layer8', label='best', save_maps=True)

#Problem ist hier nur mehr die richtigen settings zu finden im convolutional network je nach deinem input.
#Bei dem jetzigen input von 3x28x28 kannst du nicht:

#(2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
#(5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
#(12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)

#rennen, da du sonst mit 192x0x0 hinunterpoolst.
#fuer ein groesseres dataset, mit e.g 3x224x224 ginge das eher.

_ = model(batch[0].to(device))

RuntimeError: ignored

In [ ]:
""" BIS HIER IST ALLES GUT. DU musst lediglich die input dimensionen aendern.
    DAS GANZE NETZWERK HAT DIE RICHTIGEN DATATYPES NUN ALLERDINGS IST DER INPUT NUN UNPASSEND WEIL DU MIT KLEINEREN BILDERN DIE STEPS IM CONVOLUTIONAL FORWARD PASS AENDERN MUSST
    BEI DEN KLEINEN BILDERN WIRD DAS GANZE AUF 192x0x0 gepooled. viel zu klein also.
    """

In [ ]:
train_set, val_set = get_skin_loader("hmnist_28_28_RGB.csv", train=True, batch_size=50, num_workers=0, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# get some random training images
dataiter = iter(train_set)
images, labels = next(dataiter)

#images and labels are on CPU! should be on cuda 
images.device
labels.device


number of images: 10015
images loaded and resized
images normalized
int32
<class 'numpy.ndarray'>
images zipped


device(type='cpu')

In [ ]:
def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()

In [ ]:
class project2(nn.Module):
    def __init__(self, model_trained: nn.Module, num_classes: int = 10):
        
        super(project2, self).__init__()
        self.num_classes = num_classes
        self.classifier = nn.Linear(1000, num_classes)
        self.model_trained = model_trained

    def forward(self, x):
        x = self.model_trained(x)
        x = self.classifier(x)
        return x

In [ ]:
def _forward(network: nn.Module, data: DataLoader, metric: callable, give_logits=False, augment_data=False):
    device = next(network.parameters()).device
    
    transform = transforms.Compose([
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomResizedCrop(size=(28, 28), scale=(0.7, 1)),
    ])
    
    #if give_logits:
    for x, y in data:
        if augment_data and np.random.uniform()>0.5:
            x = transform(x)
        x = np.repeat(np.repeat(x.cpu(), repeats=8, axis=2), repeats=8, axis=3)

        x, y = x.to(device, dtype=torch.float), y.to(device, dtype=torch.long)
        

        logits = network(x)
        res = metric(logits, y)
        if give_logits:
            yield res, torch.argmax(logits.detach().clone(), dim=1).tolist(), y.detach().clone().tolist()
        else:
            yield res
    #else:
        #for x, y in data:
            #x = np.repeat(np.repeat(x, repeats=8, axis=2), repeats=8, axis=3)
            #x, y = x.to(device, dtype=torch.float), y.to(device, dtype=torch.long)
            #logits = network(x)
            #res = metric(logits, y)
            #yield res

@torch.no_grad()
def evaluate(network: nn.Module, data: DataLoader, metric: callable, give_logits=False) -> list:
    network.eval()
    if give_logits:
        yielded_results = _forward(network, data, metric, give_logits=True)        
        return [(res[0].item(), res[1], res[2]) for res in yielded_results]
    else:
        results = _forward(network, data, metric)
        return [res.item() for res in results]

In [ ]:
@torch.enable_grad()
def update(network: nn.Module, data: DataLoader, loss: nn.Module,
           opt: optim.Optimizer, augment_data=True) -> list:
    network.train()
    
    errs = []
    for err in _forward(network, data, loss, augment_data=augment_data):
        errs.append(err.item())
        opt.zero_grad()
        err.backward()
        opt.step()
        
    return errs

In [ ]:
@torch.no_grad()
def accuracy(logits, targets):

    top_pred = logits.argmax(1, keepdim = True)
    correct = top_pred.eq(targets.view_as(top_pred)).sum()
    acc = correct.float() / targets.shape[0]

    return acc

In [ ]:
model_pretrained = models.alexnet(weights=True)
# model_pretrained = models.resnet50(pretrained=True)
#model_pretrained = models.resnet152(pretrained=True)
for param in model_pretrained.parameters():
    param.requires_grad = False

model_project2 = project2(model_pretrained, num_classes = 7)
model_project2.eval()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


project2(
  (classifier): Linear(in_features=1000, out_features=7, bias=True)
  (model_trained): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
   

In [ ]:
def generate_data(model, epochs:int, lr:float, use_augmented_data:bool, model_name:str,
                  train_set=train_set, val_set=val_set, device=device, to_folder='epoch_data_all_samples'):
    
    #summaries are going to be stored in a pd.DataFrame
    df_data = pd.DataFrame(columns=['name', 'epoch', 'lr', 'classifier.weight', 'classifier.bias',
                                'confusion_matrix', 'train_errs', 'valid_errs'])
    
    #push to GPU
    model_project2.to(device)
    #push to GPU 
    device = next(model_project2.parameters()).device

    # optimiser + loss function
    optim = torch.optim.Adam(model_project2.parameters(), lr = lr)
    ce = nn.CrossEntropyLoss()

    name_list, epoch_list, lr_list = [], [], []
    conf_matrix_list, train_errs_list, valid_errs_list = [], [], []

    
    weights_shape = np.zeros_like(dict([i for i in model_project2.named_parameters()])[
        'classifier.weight'].detach().cpu().numpy()).shape


    bias_shape = np.zeros_like(dict([i for i in model_project2.named_parameters()])[
        'classifier.bias'].detach().cpu().numpy()).shape


    weights_array, bias_array = np.zeros((epochs,weights_shape[0],weights_shape[1]), dtype=np.float32), np.zeros((epochs,bias_shape[0]), dtype=np.float32)

    #delete again
    logits_argmax_list = None
    y_list = None
    val_errs = None
    conf_matrix = None

    for i in trange(epochs):
        df_data = pd.DataFrame(columns=['name', 'epoch', 'lr', 'classifier.weight', 'classifier.bias',
                                    'confusion_matrix', 'train_errs', 'valid_errs', 'used_data_augmentation'])
        start_time = time.time()

        # training 
        local_errs = update(model_project2, train_set, ce, optim, augment_data=True)
        # training
        train_errs_list.append(sum(local_errs) / len(local_errs))

        # validation
        val_output = np.array(evaluate(model_project2, val_set, ce, give_logits=True))
        val_errs = val_output[:,0]
        valid_errs_list.append((sum(val_errs) / len(val_errs)))

        # logits and y for confusion matrix
        logits_argmax_list = val_output[:,1]
        y_list = val_output[:,2]
        conf_matrix = confusion_matrix(np.array(sum(y_list.tolist(), [])), np.array(sum(logits_argmax_list.tolist(), [])))
        conf_matrix_list.append(conf_matrix)

        acc_train = evaluate(model_project2, train_set, metric = accuracy)
        acc_val = evaluate(model_project2, val_set, metric = accuracy)

        name_list.append('alexnet_1')
        epoch_list.append(i)
        lr_list.append(lr)

        # add model parameters, but only the last layer
        weights_array[i] = dict([i for i in model_project2.named_parameters()])['classifier.weight'].detach().cpu().numpy()
        bias_array[i] = dict([i for i in model_project2.named_parameters()])['classifier.bias'].detach().cpu().numpy()

        # safe the data for each epoch
        df_data['name'], df_data['epoch'], df_data['lr'] = name_list, epoch_list, lr_list
        df_data['classifier.weight'],df_data['classifier.bias'],df_data['confusion_matrix'] = list(weights_array)[:i+1], list(bias_array)[:i+1], conf_matrix_list
        df_data['train_errs'], df_data['valid_errs'] = train_errs_list, valid_errs_list
        df_data['used_data_augmentation'] = [use_augmented_data]*len(df_data)
        
        # safe current dataframe after each epoch
        df_data.to_pickle(f'{to_folder}\\{model_name}_epoch_{i+1}')
        
        print("Accuracy on train data: {:.2f} %".format(sum(acc_train) / len(acc_train) * 100))
        print("Accuracy on validation data: {:.2f} %".format(sum(acc_val) / len(acc_val) * 100))
        print(f'--- {round(time.time()-start_time, 2)} seconds ---')
    
    return df_data

In [ ]:
df_alexnet_1 = generate_data(model_pretrained, epochs=3, lr=1e-4, use_augmented_data=False, model_name='alexnet_1')

  0%|          | 0/3 [00:00<?, ?it/s]

<built-in method type of Tensor object at 0x7f1f61ea7860> <built-in method type of Tensor object at 0x7f1f61ea71d0>


RuntimeError: ignored

#### Shape of image as an input to the network has to be: ([3, 224, 224]). For now it should be ([3, 28, 28]), because we change the dimension function of the image in the function _forward().

#### Defined the function to generate data:

In [ ]:
def generate_data(model, epochs:int, lr:float, use_augmented_data:bool, model_name:str,
                  train_set=train_set, val_set=val_set, device=device, to_folder='epoch_data_all_samples'):
    
    df_data = pd.DataFrame(columns=['name', 'epoch', 'lr', 'classifier.weight', 'classifier.bias',
                                'confusion_matrix', 'train_errs', 'valid_errs'])
    
    model_project2.to(device)
    device = next(model_project2.parameters()).device
    print(device)
    # optimiser + loss function
    optim = torch.optim.Adam(model_project2.parameters(), lr = lr)
    ce = nn.CrossEntropyLoss()

    name_list, epoch_list, lr_list = [], [], []
    conf_matrix_list, train_errs_list, valid_errs_list = [], [], []

    weights_shape = np.zeros_like(dict([i for i in model_project2.named_parameters()])[
        'classifier.weight'].detach().cpu().numpy()).shape


    bias_shape = np.zeros_like(dict([i for i in model_project2.named_parameters()])[
        'classifier.bias'].detach().cpu().numpy()).shape


    weights_array, bias_array = np.zeros((epochs,weights_shape[0],weights_shape[1]), dtype=np.float32), np.zeros((epochs,bias_shape[0]), dtype=np.float32)

    #delete again
    logits_argmax_list = None
    y_list = None
    val_errs = None
    conf_matrix = None

    for i in trange(epochs):
        df_data = pd.DataFrame(columns=['name', 'epoch', 'lr', 'classifier.weight', 'classifier.bias',
                                    'confusion_matrix', 'train_errs', 'valid_errs', 'used_data_augmentation'])
        start_time = time.time()

        # training 
        local_errs = update(model_project2, train_set, ce, optim, augment_data=True)
        # training
        train_errs_list.append(sum(local_errs) / len(local_errs))

        # validation
        val_output = np.array(evaluate(model_project2, val_set, ce, give_logits=True))
        val_errs = val_output[:,0]
        valid_errs_list.append((sum(val_errs) / len(val_errs)))

        # logits and y for confusion matrix
        logits_argmax_list = val_output[:,1]
        y_list = val_output[:,2]
        conf_matrix = confusion_matrix(np.array(sum(y_list.tolist(), [])), np.array(sum(logits_argmax_list.tolist(), [])))
        conf_matrix_list.append(conf_matrix)

        acc_train = evaluate(model_project2, train_set, metric = accuracy)
        acc_val = evaluate(model_project2, val_set, metric = accuracy)

        name_list.append('alexnet_1')
        epoch_list.append(i)
        lr_list.append(lr)

        # add model parameters, but only the last layer
        weights_array[i] = dict([i for i in model_project2.named_parameters()])['classifier.weight'].detach().cpu().numpy()
        bias_array[i] = dict([i for i in model_project2.named_parameters()])['classifier.bias'].detach().cpu().numpy()

        # safe the data for each epoch
        df_data['name'], df_data['epoch'], df_data['lr'] = name_list, epoch_list, lr_list
        df_data['classifier.weight'],df_data['classifier.bias'],df_data['confusion_matrix'] = list(weights_array)[:i+1], list(bias_array)[:i+1], conf_matrix_list
        df_data['train_errs'], df_data['valid_errs'] = train_errs_list, valid_errs_list
        df_data['used_data_augmentation'] = [use_augmented_data]*len(df_data)
        
        # safe current dataframe after each epoch
        df_data.to_pickle(f'{to_folder}\\{model_name}_epoch_{i+1}')
        
        print("Accuracy on train data: {:.2f} %".format(sum(acc_train) / len(acc_train) * 100))
        print("Accuracy on validation data: {:.2f} %".format(sum(acc_val) / len(acc_val) * 100))
        print(f'--- {round(time.time()-start_time, 2)} seconds ---')
    
    return df_data

#### Generate data for alexnet: epochs=70, lr=1e-4, use_augmented_data=False;

In [ ]:
model_pretrained = models.alexnet(pretrained=True)
# model_pretrained = models.resnet50(pretrained=True)
#model_pretrained = models.resnet152(pretrained=True)
for param in model_pretrained.parameters():
    param.requires_grad = False

model_project2 = project2(model_pretrained, num_classes = 7)
model_project2.eval()

df_alexnet_1 = generate_data(model_pretrained, epochs=3, lr=1e-4, use_augmented_data=False, model_name='alexnet_1')

cuda:0


  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
from medcam import medcam

model = medcam.inject(model_project2, output_dir='attention_maps', backend='gcam', layer='layer8', label='best', save_maps=True)

In [ ]:
batch = next(iter(train_set))

_ = model(batch[0])

<bound method Module.parameters of AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=92

RuntimeError: ignored